# Import Libraries

In [28]:
# basic imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# ML libraries
from sklearn.model_selection import train_test_split

# remove warnings
import warnings
warnings.filterwarnings("ignore")

# Sample Data

In [29]:
# Generate mock customer data
np.random.seed(42)
num_customers = 2000

data = {
    'CustomerID': [f'CUST{1000+i}' for i in range(num_customers)],
    'Gender': np.random.choice(['Male', 'Female'], num_customers, p=[0.5, 0.5]),
    'SeniorCitizen': np.random.choice([0, 1], num_customers, p=[0.84, 0.16]),
    'Partner': np.random.choice(['Yes', 'No'], num_customers, p=[0.48, 0.52]),
    'Dependents': np.random.choice(['Yes', 'No'], num_customers, p=[0.3, 0.7]),
    'Tenure': np.random.randint(1, 73, num_customers), # Months
    'PhoneService': np.random.choice(['Yes', 'No'], num_customers, p=[0.9, 0.1]),
    'MultipleLines': np.random.choice(['Yes', 'No', 'No phone service'], num_customers, p=[0.42, 0.48, 0.1]),
    'InternetService': np.random.choice(['DSL', 'Fiber optic', 'No'], num_customers, p=[0.34, 0.44, 0.22]),
    'OnlineSecurity': np.random.choice(['Yes', 'No', 'No internet service'], num_customers, p=[0.28, 0.50, 0.22]),
    'OnlineBackup': np.random.choice(['Yes', 'No', 'No internet service'], num_customers, p=[0.34, 0.44, 0.22]),
    'DeviceProtection': np.random.choice(['Yes', 'No', 'No internet service'], num_customers, p=[0.34, 0.44, 0.22]),
    'TechSupport': np.random.choice(['Yes', 'No', 'No internet service'], num_customers, p=[0.29, 0.49, 0.22]),
    'StreamingTV': np.random.choice(['Yes', 'No', 'No internet service'], num_customers, p=[0.38, 0.40, 0.22]),
    'StreamingMovies': np.random.choice(['Yes', 'No', 'No internet service'], num_customers, p=[0.39, 0.39, 0.22]),
    'Contract': np.random.choice(['Month-to-month', 'One year', 'Two year'], num_customers, p=[0.55, 0.24, 0.21]),
    'PaperlessBilling': np.random.choice(['Yes', 'No'], num_customers, p=[0.59, 0.41]),
    'PaymentMethod': np.random.choice(['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)'], num_customers, p=[0.34, 0.23, 0.22, 0.21]),
    'MonthlyCharges': np.random.normal(loc=65, scale=30, size=num_customers).clip(18, 120).round(2),
}

# create DataFrame
df_customers = pd.DataFrame(data)

# Generate TotalCharges based on Tenure and MonthlyCharges with some noise
df_customers['TotalCharges'] = (df_customers['Tenure'] * df_customers['MonthlyCharges'] * np.random.uniform(0.95, 1.05, num_customers)).round(2)

# Make some TotalCharges empty for realism (e.g., new customers with 0 tenure)
df_customers.loc[df_customers['Tenure'] == 1, 'TotalCharges'] = df_customers['MonthlyCharges']

# Get all customers with Tenure < 3
low_tenure_mask = df_customers['Tenure'] < 3
# Get random sample of 1% of those low-tenure customers
random_indices = df_customers[low_tenure_mask].sample(frac=0.01, random_state=42).index
# Set TotalCharges to NaN for these
df_customers.loc[random_indices, 'TotalCharges'] = np.nan

# Simulate Churn (more likely for month-to-month, higher charges, lower tenure)
churn_probability = 0.1 \
+ 0.15 * (df_customers['Contract'] == 'Month-to-month') \
+ 0.1 * (df_customers['InternetService'] == 'Fiber optic') \
+ 0.001 * (df_customers['MonthlyCharges'] - 65) \
- 0.002 * (df_customers['Tenure'] - 36) \
+ 0.1 * (df_customers['OnlineSecurity'] == 'No') \
+ 0.1 * (df_customers['TechSupport'] == 'No')

churn_probability = np.clip(churn_probability, 0.01, 0.99)
df_customers['Churn'] = np.random.binomial(1, churn_probability, num_customers).astype(str)
df_customers['Churn'] = df_customers['Churn'].replace({'1': 'Yes', '0': 'No'})

# Replace 'No phone service' and 'No internet service' for consistency
for col in ['MultipleLines']:
    df_customers[col] = df_customers.apply(lambda row: 'No' if row['PhoneService'] == 'No' else row[col], axis=1)

for col in ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']:
    df_customers[col] = df_customers.apply(lambda row: 'No' if row['InternetService'] == 'No' else row[col], axis=1)

# make folder if it doesn't exist
if not os.path.exists('Data'):
    os.makedirs('Data')

# Save the DataFrame to an Excel file
df_customers.to_excel('./Data/telecom_churn_mock_data.xlsx', index=False)
print("Mock telecom churn data generated: telecom_churn_mock_data.xlsx")


Mock telecom churn data generated: telecom_churn_mock_data.xlsx


# Exploratory Data Analysis

In [30]:
# load the data
df = pd.read_excel('./Data/telecom_churn_mock_data.xlsx')

print("Data loaded successfully")

Data loaded successfully


In [31]:
# show all columns
pd.set_option('display.max_columns', None)

# data preview
df.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,CUST1000,Male,0,No,No,30,Yes,Yes,No,No,No,No,No,No,No,Month-to-month,No,Bank transfer (automatic),69.55,2047.01,Yes
1,CUST1001,Female,0,No,Yes,11,Yes,Yes,Fiber optic,No internet service,No internet service,No,No internet service,No,No internet service,Month-to-month,Yes,Electronic check,48.08,522.42,Yes
2,CUST1002,Female,1,No,No,17,No,No,Fiber optic,No,Yes,No,No,Yes,No,Two year,No,Electronic check,36.56,610.07,No
3,CUST1003,Female,0,Yes,No,26,Yes,No,No,No,No,No,No,No,No,One year,Yes,Bank transfer (automatic),79.72,2159.26,No
4,CUST1004,Male,0,Yes,Yes,23,Yes,No,Fiber optic,No internet service,No,No,Yes,No,Yes,Month-to-month,Yes,Mailed check,70.42,1672.56,Yes


In [32]:
# shape of data
print(f'Rows = {df.shape[0]}')
print(f'Columns = {df.shape[1]}')

Rows = 2000
Columns = 21


In [33]:
# data info
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerID        2000 non-null   object 
 1   Gender            2000 non-null   object 
 2   SeniorCitizen     2000 non-null   int64  
 3   Partner           2000 non-null   object 
 4   Dependents        2000 non-null   object 
 5   Tenure            2000 non-null   int64  
 6   PhoneService      2000 non-null   object 
 7   MultipleLines     2000 non-null   object 
 8   InternetService   2000 non-null   object 
 9   OnlineSecurity    2000 non-null   object 
 10  OnlineBackup      2000 non-null   object 
 11  DeviceProtection  2000 non-null   object 
 12  TechSupport       2000 non-null   object 
 13  StreamingTV       2000 non-null   object 
 14  StreamingMovies   2000 non-null   object 
 15  Contract          2000 non-null   object 
 16  PaperlessBilling  2000 non-null   object 


### Observation
- Data needs some type conversion like object to category for memory effecient

In [34]:
# summary statistics
df.describe()

,SeniorCitizen,Tenure,MonthlyCharges,TotalCharges
count,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.151500,36.342500,65.880710,2394.132725
std,0.358625,21.029392,27.154716,1807.861046
min,0.000000,1.000000,18.000000,18.000000
25%,0.000000,18.000000,45.342500,934.712500
50%,0.000000,37.000000,65.460000,1972.770000
75%,0.000000,54.000000,84.992500,3532.675000
max,1.000000,72.000000,120.000000,8756.020000


In [35]:
# column names
df.columns

Index(['CustomerID', 'Gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'Tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

## Duplicate Rows

In [36]:
# get count of duplicate rows
print(f'Number of duplicate rows: {df.duplicated().sum()}')

# display duplicate rows if any exist
if df.duplicated().sum() > 0:
    print("\nDuplicate rows:")
    print(df[df.duplicated()])


Number of duplicate rows: 0


## Missing Values

In [37]:
# check missing values
missing_values = df.isnull().sum()
missing_percentages = (missing_values / len(df)) * 100

# create a summary DataFrame
missing_data = pd.DataFrame({
    'Missing Values': missing_values,
    'Missing Percentage': missing_percentages.round(2)
})

# display only columns with missing values
print("Columns with missing values:")
print(missing_data[missing_data['Missing Values'] > 0])

# if no missing values, print a message
if missing_data['Missing Values'].sum() == 0:
    print("\nNo missing values found in the dataset.")

Columns with missing values:
Empty DataFrame
Columns: [Missing Values, Missing Percentage]
Index: []

No missing values found in the dataset.


## Type Casting

In [38]:
# Get memory usage for each column
memory_usage = df.memory_usage(deep=True).sum() / 1024  # Convert bytes to KB

print(f"Total memory usage before type conversion: '{memory_usage:.2f}' KB")

Total memory usage before type conversion: '2144.06' KB


In [42]:
# datatype before type casting
df.dtypes

CustomerID          category
Gender              category
SeniorCitizen          int64
Partner             category
Dependents          category
Tenure                 int64
PhoneService        category
MultipleLines       category
InternetService     category
OnlineSecurity      category
OnlineBackup        category
DeviceProtection    category
TechSupport         category
StreamingTV         category
StreamingMovies     category
Contract            category
PaperlessBilling    category
PaymentMethod       category
MonthlyCharges         int64
TotalCharges           int64
Churn               category
dtype: object

In [41]:
# Convert object columns to category
object_columns = df.select_dtypes(include=['object']).columns
for col in object_columns:
    df[col] = df[col].astype('category')

# Convert float columns to int where appropriate    
df['MonthlyCharges'] = df['MonthlyCharges'].astype('int64')
df['TotalCharges'] = df['TotalCharges'].astype('int64')

# check datatype
df.dtypes

CustomerID          category
Gender              category
SeniorCitizen          int64
Partner             category
Dependents          category
Tenure                 int64
PhoneService        category
MultipleLines       category
InternetService     category
OnlineSecurity      category
OnlineBackup        category
DeviceProtection    category
TechSupport         category
StreamingTV         category
StreamingMovies     category
Contract            category
PaperlessBilling    category
PaymentMethod       category
MonthlyCharges         int64
TotalCharges           int64
Churn               category
dtype: object

In [40]:
# Get memory usage for each column
memory_usage = df.memory_usage(deep=True).sum() / 1024  # Convert bytes to KB

print(f"Total memory usage after type conversion: '{memory_usage:.2f}' KB")


Total memory usage after type conversion: '293.73' KB


### **When to use Category**
Use the category dtype in pandas when a column contains repeated string values with a limited number of unique entries (i.e., low cardinality), such as Gender, PhoneService, MultipleLines, or InternetService. This is because category stores the data more efficiently by assigning integer codes to each unique value and referencing them via a lookup table, which significantly reduces memory usage and can speed up operations like filtering, grouping, and sorting. However, avoid using category for columns with mostly unique values—such as names, emails, or user IDs—because the categorical structure adds overhead without providing memory savings or performance improvements. In such cases, keeping the column as object is more practical and flexible